<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tabular-Output" data-toc-modified-id="Tabular-Output-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tabular Output</a></span></li></ul></div>

# Tabular Output

In addition to waveforms, `myhdlpeek` also lets you display the captured traces as tables.
To demonstrate, I'll use our old friend the multiplexer:

In [1]:
from myhdl import *
from myhdlpeek import Peeker  # Import the myhdlpeeker module.

def mux(z, a, b, sel):
    """A simple multiplexer."""
    @always_comb
    def mux_logic():
        if sel == 1:
            z.next = a  # Signal a sent to mux output when sel is high.
        else:
            z.next = b  # Signal b sent to mux output when sel is low.
    return mux_logic

# Create some signals to attach to the multiplexer.
a, b, z = [Signal(0) for _ in range(3)]  # Integer signals for the inputs & output.
sel = Signal(intbv(0)[1:])               # Binary signal for the selector.

# Create some Peekers to monitor the multiplexer I/Os.
Peeker.clear()         # Clear any existing Peekers. (Start with a clean slate.)
Peeker(a, 'a')         # Add a Peeker to the a input.
Peeker(b, 'b')         # Add a Peeker to the b input.
Peeker(z, 'z')         # Add a peeker to the z output.
Peeker(sel, 'select')  # Add a Peeker to the select input. The Peeker label doesn't have to match the signal name.

# Instantiate mux.
mux_1 = mux(z, a, b, sel)

# Apply random patterns to the multiplexer.
from random import randrange
def test():
    for _ in range(8):
        a.next, b.next, sel.next = randrange(8), randrange(8), randrange(2)
        yield delay(1)

# Simulate the multiplexer, testbed and the peekers.
sim = Simulation(mux_1, test(), *Peeker.instances()).run()

<class 'myhdl.StopSimulation'>: No more events


Once the simulation has run, I can display the results using a table instead of waveforms:

In [2]:
Peeker.to_html_table()

Time,a,b,select,z
0,2,2,1,2
1,4,5,0,5
2,0,4,0,4
3,5,5,0,5
4,0,6,1,0
5,6,1,1,6
6,2,5,1,2
7,0,0,0,0


I can use the same options for tabular output that are available for showing waveforms: 

In [3]:
Peeker.to_html_table('select a b z', start_time=3) # Select and change order of signals, and set start time.

Time,select,a,b,z
3,0,5,5,5
4,1,0,6,0
5,1,6,1,6
6,1,2,5,2
7,0,0,0,0


There's even a version for use in console mode (outside of the Jupyter environment):

In [4]:
Peeker.to_text_table('select a b z')

  Time    select    a    b    z
------  --------  ---  ---  ---
     0         1    2    2    2
     1         0    4    5    5
     2         0    0    4    4
     3         0    5    5    5
     4         1    0    6    0
     5         1    6    1    6
     6         1    2    5    2
     7         0    0    0    0
